In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
POINTS = 500

In [3]:
def lagrange_interpolation(X, Y, x):
    result = 0
    n = len(X)
    for i in range(n):
        li = Y[i]
        for j in range(n):
            if i != j:
                li *= (x - X[j]) / (X[i] - X[j])
        result += li
    return result

In [4]:
def calculate_coefficients(X, Y):
    n = len(X)
    A = np.empty((n, n))
    for i in range(n):
        A[i][0] = Y[i]
    for j in range(1,n):
        for i in range(n-j):
            A[i][j] = (A[i+1][j-1] - A[i][j-1]) / (X[i+j] - X[i])
    return A[0]

In [5]:
def newton_interpolation(X, Y, x):
    result = 0
    n = len(X)
    A = calculate_coefficients(X, Y)
    for i in range(n):
        product = A[i]
        for j in range(i):
            product *= (x - X[j])
        result += product
    return result

In [6]:
def chebyshev_roots(n):
    return np.flip(np.fromfunction(lambda i: np.cos(np.pi * (i + 0.5) / n), (n, )))

In [7]:
def chebyshev_nodes(a, b, n):
    if a > b:
        a, b = b, a
    roots = chebyshev_roots(n)
    stretch = (b - a) / 2
    return (roots + 1) * stretch + a

In [16]:
def f(x):
    return np.sin(2*x) * np.sin(x**2/np.pi)

In [17]:
def f_1(x):
    return 2 * np.cos(2*x) * np.sin(x**2/np.pi) + (2*x / np.pi) * np.sin(2*x) * np.cos(x**2/np.pi)

In [9]:
def interpolate_lagrange(a, b, n):
    x_inter = np.linspace(a, b, n)
    x_print = np.linspace(a, b, POINTS)
    y_inter = f(x_inter)
    y_print = f(x_print)
    p_print = np.empty(POINTS)
    for i in range(POINTS):
        p_print[i] = lagrange_interpolation(x_inter, y_inter, x_print[i])
    return x_inter, x_print, y_inter, y_print, p_print

In [10]:
def print_interpolation_plots_lagrange_ch(n):
    X_inter = chebyshev_nodes(-np.pi, 2*np.pi, n)
    X_print = np.linspace(-np.pi, 2*np.pi, 500)
    Y_inter = f(X_inter)
    Y_print = f(X_print)
    P_print = np.empty(500)
    for i in range(500):
        P_print[i] = lagrange_interpolation(X_inter, Y_inter, X_print[i])
    print('Liczba węzłów:', n)
    print('Błąd interpolacji(norma euklidesowa):', np.linalg.norm(Y_print-P_print) / 500)
    print('Błąd interpolacji(norma maksimum):', np.linalg.norm(Y_print-P_print, ord=np.inf))
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.plot(X_print, P_print, label="W(x)")
    ax.plot(X_print, Y_print, label="f(x)")
    ax.plot(X_inter, Y_inter, '*', label='Węzły interpolacji')
    ax.legend()
    plt.show()

In [11]:
def interpolate_newton(a, b, n):
    X_inter = np.linspace(a, b, n)
    X_print = np.linspace(a, b, POINTS)
    Y_inter = f(X_inter)
    Y_print = f(X_print)
    P_print = np.empty(POINTS)
    for i in range(POINTS):
        P_print[i] = newton_interpolation(X_inter, Y_inter, X_print[i])
    return X_inter, X_print, Y_inter, Y_print, P_print

In [12]:
def print_errors(Y_print, P_print):
    print('Błąd interpolacji(norma euklidesowa):', np.linalg.norm(Y_print-P_print) / POINTS)
    print('Błąd interpolacji(norma maksimum):', np.linalg.norm(Y_print-P_print, ord=np.inf))

In [13]:
def print_plots(x_inter, x_print, y_inter, y_print, p_print):
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.plot(x_print, p_print, label="W(x)")
    ax.plot(x_print, y_print, label="f(x)")
    ax.plot(x_inter, y_inter, '*', label='Węzły interpolacji')
    ax.set_title('elo')
    ax.set_ylabel('y')
    ax.set_xlabel('x')
    ax.legend()
    plt.show()

In [14]:
def exercise_newton(a, b, n):
    print('Liczba węzłów:', n)
    X_inter, X_print, Y_inter, Y_print, P_print = interpolate_newton(a, b, n)
    print_errors(Y_print, P_print)
    print_plots(X_inter, X_print, Y_inter, Y_print, P_print)

In [63]:
def exercise_lagrange(a, b, n):
    print('Liczba węzłów:', n)
    X_inter, X_print, Y_inter, Y_print, P_print = interpolate_lagrange(a, b, n)
    print_errors(Y_print, P_print)
    print_plots(X_inter, X_print, Y_inter, Y_print, P_print)

Exercise2

In [62]:
def calculate_hermite_coefficients(X, f, f_1):
    n = 2 * len(X)
    x = np.empty(n)
    for i in range(n):
        x[i] = X[(i//2)]
    A = np.empty((n, n))
    for i in range(n):
        A[i][0] = f(x[i])
    for i in range(0, n, 2):
        A[i][1] = f_1(x[i])
    for i in range(1, n-1, 2):
        A[i][1] = (A[i+1][0] - A[i][0]) / (x[i+1] - x[i])
    for j in range(2,n):
        for i in range(n - j):
            A[i][j] = (A[i+1][j-1] - A[i][j-1]) / (x[i+j] - x[i])
    return A[0]

In [64]:
def hermite_interpolation(X, f, f_1, x):
    result = 0
    n = 2 * len(X)
    A = calculate_hermite_coefficients(X, f, f_1)
    for i in range(n):
        product = A[i]
        for j in range(i):
            product *= (x - X[j])
        result += product
    return result

[2. 3. 1. 2.]


In [43]:
len(np.array([1,2]))

2